# GPT architecture

## Transformer architecture


First off, we have the inputs and outputs (shifted right). goes througth the embeddings (POS encoding). All this passes through number of layers. lets take 4 layers, So, inputs will go through 4 encoders and then outputs and final encoder values passes through the 4 decoders. then the last decoder will do a transformation and pass through the softmax and then give probability sampling generation

Second off, in each encoder there going to be four process:

- Mulihead attention
  
  There are a bunch of heads learning different semantic info from a unique perspective and all the heads have different leanable parameters. Lets see the inner workings :

     - Keys, Query, Value
       
       Basically, key is like all the words in a sentence and query is like the word trying to find and when they dot product and gives a attention score.

     - Scaled Dot-product attention (n heads all run in parallel) 

       Lets see how the one head works, first the k and q is dot product and its attention score is formed and also these values are scaled so we dont get high values and then sometimes we do torch trill masking and then we do softmax for getng more confidence in the attention scoore and then we do matrix multiply with the v value which a output of input vector values and attention placed on each token.

     - Concatenate results
     - Linear

- Residual connection (add) then normalize

- Feed Forward (Linear -> ReLU -> Linear)

- Residual connection then normalize

Feeds into the next encoder block. if its the last encoder block it feed into each decoder block

multi-head atention - we look at past, present and future
masked multi-head attention - we look at past and present only


## Self-Attention working

So, lets take a four token sequence "My dog has fleas". We are going to hihlight which words are going to correlating eachother or attention mechanism is going multipy them together and get a high score. lets see how GPT understands from the inside.

	     My	 dog	 has	fleas  
My	   low	 med	 low	  low
dog	   med   low   med    high
has	   low   med   low    med
fleas	 low   high  med    low

So from these you can see high and low values, which makes the network learn how to place the attention score on the tokens and thses learn through embeddings. Attention is used to generate tokens, thats how the gpt works.

## GPT architecture

Basically gpt works same as the transformer architecture but the encoder part is just removed.

Also in attention we use scaling in the attention scores so we can protect the model from vanishing gradient problem doesnt attack.

Let me explian it onw more time:

GPT process :

- Tokenized outputs
- Embeddings + POS encoding
- n Decoders according to the layers
    In each decoder the process is :
       - Multihead attention
           n heads of multihead runs in parallel.
            the multihead process is :
          - Keys, Query and Value
          - Scaled dot product attention (head)
                   Basically, in each head the key and query will be done dot product and then its scaled to combat the vanishig gradient after that torch.trill masking for not lookin into the future and then softmax then the value and softmax will matrix multiply. the output will be a blend of inputs vector and attention placed on each token.
          - Conconate results
          - Linear
       - Residual connection (Add) then normalise
       - Feed Forward  (Linear -> ReLU -> Linear)
       - Residual connection (Add) and normalise
- nnLinear
- Softmax
- Probability sampling generation
- Compare targets to outputs -> backpropogate


## GPT Code

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250
n_embd = 384
n_layer = 4

In [3]:
with open('Book.txt','r',encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [5]:
n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [ ]:
@torch.no_grad   #this is used to make the computation easier by removing gradients

def estimate_loss():
    out = {}
    model.eval()  # to make the model enter the evaluation mode
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits , loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

in the gpt model we have to do positional encoding to the tokenized inputs, but if we put the positional encoding its a fixed function. which does not learn about the data. so we going to put the positional encoding inside the nn.embedding.

self.blocks is the variable containing the sequential running of the decoders of 4 layer using the n_layers
LayerNOrm is used to end of the netowrk to make the model to converge better and normalize
linear it there so it can make the final outpu workable with the softmax

while doing in practice sinisudal embeddings are used to transformer models and while doing GPT they use learnable embeddings

im the forawrd pass, we are going to add a code where we are going to create token embedding and positional embedding and add them both and then run blocks on then do layernorm and then do the linear function

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)  # we are creating a embedding table with dimentional of vocab_size and n_embed
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)  #we are creating embedding table for positional encoding using the block size and n_embed
        self.blocks = nn.Sequential(*(Block(n_embd, n_head=n_head) for in range(n_layer))) 
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self, _init_weights)
    
    def _init_weights(self,module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)



    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.positional_embedding_table(torch.arrange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B - Batch, T - Time, C - Channel (size of sequence)
            logits = logits.view(B*T,C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)  # we call the forward pass
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)   #we get the probability distribution and dimension is -1 because we want the +1 index expected index
            index_next = torch.multinomial(probs, num_samples=1) #takes the highest number of prob
            index = torch.cat((index, index_next),dim=1) # concatinates to the next element the whole size
        return index
    
model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) # this basically the index in the above generate parameter, which is a single zero
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, loss: {losses}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())